# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20902046


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/39 [00:00<?, ?it/s]

Rendering models:   8%|▊         | 3/39 [00:01<00:21,  1.65it/s]

Rendering models:  10%|█         | 4/39 [00:05<00:54,  1.57s/it]

Rendering models:  13%|█▎        | 5/39 [00:05<00:40,  1.19s/it]

Rendering models:  15%|█▌        | 6/39 [00:06<00:30,  1.08it/s]

...

Rendering models:  90%|████████▉ | 35/39 [00:16<00:01,  3.05it/s]

Rendering models:  92%|█████████▏| 36/39 [00:17<00:00,  3.22it/s]

Rendering models:  95%|█████████▍| 37/39 [00:17<00:00,  3.51it/s]

Rendering models:  97%|█████████▋| 38/39 [00:17<00:00,  2.91it/s]

kayleebug2017                          0.001127
nickoftona                             0.000403
Linda_J._Berkel                        0.000211
equidad1                               0.074850
not-logged-in-05ca4a4e98bc7b8f1b53     0.008870
DrJackie                               0.004242
Thedunkmasta                           0.002353
Thedunkmasta                           0.004128
Thedunkmasta                           0.003756
Thedunkmasta                           0.003142
Thedunkmasta                           0.004007
Thedunkmasta                           0.004259
Thedunkmasta                           0.003703
not-logged-in-8f952918713304e53755     0.000327
Thedunkmasta                           0.002181
Thedunkmasta                           0.004108
rcbeck                                 0.000230
not-logged-in-61a0765eeab9a5074a2a     0.000232
not-logged-in-4ec54f98b3c226b9b959     0.064822
not-logged-in-5f30b2bb9b7f9d49da41     0.004312
not-logged-in-5f30b2bb9b7f9d49da41     0

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())